In [1]:
require 'open-uri'
require 'json'
require 'daru'
require 'distribution'
require 'sqlite3'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

Object

In [2]:
dir = "#{ENV['HOME']}/cs6140/final_project"
db = SQLite3::Database.new "#{dir}/credit_risk_data.db"
db.results_as_hash = true

true

In [3]:
def p_log_p(x)
    return x * Math.log2(x)
end

# For a given data set this function returns the count for all labels
def getCount split
counts = Hash.new
split.each {
 |x| 
  if counts[x["label"]] == nil 
    counts[x["label"]] = 1.0
  elsif
    counts[x["label"]] += 1
  end
 }
 return counts
end

def entropy p
  total = p.values.inject(0, :+) + 0.0
  return -p.values.reduce(0.0) do |u, pi|
    u += p_log_p(pi/total)
  end
end

def information_gain x, splits # implemented information gain for multi label dataset
  
  rootNodeCount = getCount x
  rootEntropy   =  entropy(rootNodeCount)
  totalMembers  = rootNodeCount.values.inject(0, :+) + 0.0
  
  totalChildMembers0 = 0.0
  totalChildEntropy  = 0
  childcount0        = 0.0
  childEntropy0      = 0.0
  
  splits.each {
 |x|  
    childcount0 = getCount x
    childEntropy0 = entropy(childcount0)
    totalChildMembers0 = childcount0.values.inject(0, :+) + 0.0
    
    totalChildEntropy += ((totalChildMembers0/totalMembers)*childEntropy0)
 }
  
  #rootEntropy 
  
  
  return (rootEntropy-totalChildEntropy)
  
end

def split_on_numeric_value x, k, v # splitting based on a threshold
  firstSet = []
  secondSet = []
  splits = []
  x.each {
    |row| 
     if row["features"][k] < v # should it be less than or equals ?
      firstSet << row
    elsif
      secondSet << row
    end
  }
  return [firstSet,secondSet] 
end


def find_best_split_all_thresholds x # try all the values a feature can take to split 
  bestSplit = []
  bestInfoGain = -1.0
  fname = nil
  bestThreshold = 0 
  
  rootNodeCount = getCount x
  rootEntropy =  entropy(rootNodeCount)
  
  if rootEntropy == 0 # no need to split if all are same labels
    return
  end
  
  x[0]["features"].keys.each{
    |f|
    res = x.sort_by do |item|
      item["features"][f]
    end 
    unique_threshold = []
    

    res.each{
      |row|
      unique_threshold << row["features"][f]
    }
    
    unique_threshold = unique_threshold.uniq # only take unique values
    unique_threshold = unique_threshold.sample(40)
    (unique_threshold.length-1).times do |i|
      unique_threshold[i] = (unique_threshold[i] +unique_threshold[i+1] )/2.0 # find thresholds btw uniquie sorted members
    end
   # parent entropy ?
   
    unique_threshold.each{
      |row|
          infoGain = information_gain res,  split_on_numeric_value(res, f, row)
    if bestInfoGain == -1
      bestInfoGain = infoGain
      bestSplit = split_on_numeric_value(res, f, row)
      fname = f
      bestThreshold = row
    elsif bestInfoGain < infoGain
      bestInfoGain = infoGain
      bestSplit = split_on_numeric_value(res, f, row)
      fname = f
      bestThreshold = row
    end
      }
    
  }


  
  return [bestSplit, fname , bestInfoGain , bestThreshold]
end


#####START - supporting functions to form tree
def largest_hash_key(hash)
 return hash.max_by{|k,v| v}[0]
end
def smallest_hash_value(hash)
 return hash.min_by{|k,v| v}[1]
end
def largest_hash_value(hash)
 return hash.max_by{|k,v| v}[1]
end
def getErrorRatio(set)
  if set.length == 0
    return 0
  end
  distribution = getCount set
  return smallest_hash_value(distribution) / distribution.values.inject(0, :+) 
end
#####END - supporting functions to form tree
 

# function to build a tree based on binay splits
def form_tree dataSet, depth, maxDepth, errorRatio
  #### START - exit conditions
  if depth > maxDepth
    return
  end
  if dataSet == nil
    return
  end 
    if dataSet.length < 4 ## used a fixed min number of node members. need to make it as function arg
      return
    end
  if (getErrorRatio(dataSet) < errorRatio) && (depth>3)
    return
  end
  #### END - exit conditions
  node = {}
  node["isTerminal"] = 0
  node ["depth"] = depth
  bestSplit, fname,bestInfoGain,threshold = find_best_split_all_thresholds dataSet
  if bestInfoGain == nil || fname == nil
    return
  end
  node["splitfeature"] = fname
  node["threshold"] = threshold
  node["distribution"] = getCount dataSet

  child = []
  if bestSplit != nil
    bestSplit.each{
      |s|
      temp = form_tree s,depth+1,maxDepth,errorRatio
      if(temp != nil) ## incase the next recursive call returns nil
        child << temp
      else 
        terminalNode = {}
        terminalNode["isTerminal"] = 1
        terminalNode["depth"] = depth+1
        terminalNode["distribution"] = getCount s
        debug_count = getCount s
     #   terminalNode ["decision"] = largest_hash_key(terminalNode["distribution"])
         terminalNode ["decision"] = (debug_count[1]).to_f / ( (debug_count[0]).to_f + (debug_count[1]).to_f)
        child << terminalNode
      end  
    }
  end   
  node["children"] = child 
  return node
end

def split_on_categorical_value x, k # multiway split on category
  oneHotEncode = Hash.new
  count = 0 
  x.each {
    |row| 
     if ! (oneHotEncode.has_key?  row["features"][k])
      oneHotEncode[row["features"][k]] = count
      count += 1
    end
  }
   splits = Array.new(count) { Array.new() }
    x.each {
    |row| 
      splits[oneHotEncode[row["features"][k]]] << row
  }
  return splits
end

def find_best_split_categorical x # multiway best catagorical splits
  bestSplit = []
  bestInfoGain = -1
  fname = nil
  
  count = getCount x
  setEntropy = entropy count
  
  if setEntropy == 0
    return 
  end
  
  x[0]["features"].keys.each{
    |f|
      split = split_on_categorical_value(x,f)
     infoGain = information_gain x, split
  if bestInfoGain == -1
    bestInfoGain = infoGain
    bestSplit = split
    fname = f
    elsif bestInfoGain < infoGain
      bestInfoGain = infoGain
      bestSplit = split
      fname = f
    end
  }
  return [bestSplit, fname , bestInfoGain]
end


def printTree node
  if node == nil
    return 
  end
  depth = node["depth"] 
  temp =" "
  for i in 1..depth*5 do
    temp = temp+" "
  end 
  if node["isTerminal"] == 0
    puts "#{temp} terminal node : False"
    puts "#{temp} split feature : #{node["splitfeature"]}"
    puts "#{temp} threshold : #{node["threshold"]}"
    puts "#{temp} depth : #{node["depth"]}"
    if  (node.has_key?  "valuetoindex")
        puts "#{temp} valuetoindex : #{node["valuetoindex"]}"
    end
    puts "#{temp} distribution : #{node["distribution"]}"
  else 
    puts "#{temp} terminal node : True"
    if  (node.has_key?  "category")
        puts "#{temp} catergory : #{node["category"]}"
    end
    if  (node.has_key?  "value")
        puts "#{temp} value : #{node["value"]}"
    end
    puts "#{temp} decision : #{node["decision"]}"
    puts "#{temp} depth : #{node["depth"]}"
    puts "#{temp} distribution : #{node["distribution"]}"
  end
  puts " "  
  if node["children"] != nil
    node["children"].each{
        |child|
        printTree child
      }
  end 
  return 
end

def getDecision root, sample
  if root["isTerminal"] == 1
    return root["decision"]
  else
    if sample["features"][root["splitfeature"]] < root["threshold"]
      return getDecision root["children"][0],sample
    else
      return getDecision root["children"][1],sample
    end
  end
end
def plot x, y
  Daru::DataFrame.new({x: x, y: y}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
    plot.x_label "X"
    plot.y_label "Y"
  end
end


:plot

In [4]:
  begin db.execute("drop table batch_sgd"); rescue; end
  sql = <<SQL
    create temp table batch_sgd as 
    select *,abs(random()) % 20 as fold
    from application_train order by random() 
SQL
db.execute(sql)

[]

In [5]:
columns = db.execute("pragma table_info(batch_sgd)").map{|x| x["name"]}[1..10]

["TARGET", "NAME_CONTRACT_TYPE", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE"]

In [6]:
def random_forest columns,db
  forest = []
  (0..1).each{
    |temp|
    
  (1..19).each {
      |iter|
      batch = []
    
      rand_col = columns.sample(rand(2..3))
      rand_f = ""
      rand_col.each{
        |f|
        rand_f += f+","
        }
       rand_f += "target"
    
      db.execute("select #{rand_f} from batch_sgd where fold == #{iter}").each{ 
        |row|
          value = Hash.new {|h,k| h[k] = 0.0}
          row.keys.each{
            |key|
              if(key.is_a? String and key != "TARGET" and key != "fold")
                  if(row[key].is_a? String and row[key]!="")
                     value[row[key]] = 1
                  elsif (key != nil)
                    value[key] = (row[key] == nil) ? 0 : row[key].to_f
                  end
              end
            }
      #  puts row
          batch << { "features"=>value, "label"=>row["TARGET"]}
      }
   # puts batch[0]

      root = form_tree batch,1,5,0.01
      forest << root

    }
    }
  
  return forest
end

:random_forest

In [7]:
columns = [ "EXT_SOURCE_3", "EXT_SOURCE_2", "EXT_SOURCE_1","OCCUPATION_TYPE","DAYS_BIRTH"]
forest = random_forest columns,db
1

1

In [11]:
predict_arr = []
label_arr=[]
db.execute("select ext_source_1,ext_source_2,ext_source_3,DAYS_BIRTH,OCCUPATION_TYPE,target from batch_sgd where fold=0").each{
|row|
    value = {"EXT_SOURCE_2"=>row["EXT_SOURCE_2"].to_f,"EXT_SOURCE_3"=>row["EXT_SOURCE_3"].to_f,"EXT_SOURCE_1"=>row["EXT_SOURCE_1"].to_f}
    sample =  { "features"=>value, "label"=>row["TARGET"]}
  
    value = Hash.new {|h,k| h[k] = 0.0}
          row.keys.each{
            |key|
              if(key.is_a? String and key != "TARGET" and key != "fold")
                  if(row[key].is_a? String and row[key]!="")
                     value[row[key]] = 1
                  elsif (key != nil)
                    value[key] = (row[key] == nil) ? 0 : row[key].to_f
                  end
              end
            }
      #  puts row
    sample = { "features"=>value, "label"=>row["TARGET"]}
  
  
    y_predict = 0.0
    (0..forest.size-1).each{
      |i|
        y_predict += (getDecision forest[i], sample)
      }
    
    y_predict = y_predict/forest.size
    predict_arr << y_predict
    label_arr << row["TARGET"]
    
} 
tpr_arr,fpr_arr,auc =  plot_roc predict_arr,label_arr
puts auc
plot(fpr_arr, tpr_arr)

0.7314845685858571


#<Nyaplot::Plot:0x007fe39e942e68 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fe3a389a510 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"5e6acabf-7e06-4f8b-8e62-be004bd07b4e"}, @xrange=[0.0, 1.0], @yrange=[0.0, 1.0]>], :options=>{:x_label=>"X", :y_label=>"Y", :zoom=>true, :width=>700, :xrange=>[0.0, 1.0], :yrange=>[0.0, 1.0]}}>

## Result of the Random Forest
The accuracy is 0.7314845685858571
![ROC](RF_roc.png)